In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [2]:
import pandas as pd

df_features = pd.read_parquet("data/features_1h.parquet")
label_side = np.load("data/label_side_1h.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(44867, 11841)
(44867, 7)


4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_ac_23                        0
1h_ac_22                        0
1h_mod_stochastic               0
1h_trendflex                    0
1h_ac_31                        0
Length: 11841, dtype: int64

In [3]:
df_features = df_features[364:]
label_side = label_side[364:]

print(df_features.shape)
print(label_side.shape)

label_side

(44503, 11841)
(44503, 7)


array([[ 1.58666418e+12,  6.81173000e+03,  6.91519000e+03, ...,
         6.77566000e+03,  7.38005150e+04,  1.00000000e+00],
       [ 1.58670006e+12,  6.91519000e+03,  7.08500000e+03, ...,
         6.90500000e+03,  7.22904840e+04,  1.00000000e+00],
       [ 1.58670972e+12,  7.08500000e+03,  7.11225000e+03, ...,
         7.04632000e+03,  7.13427380e+04,  1.00000000e+00],
       ...,
       [ 1.73826948e+12,  1.05165000e+05,  1.04909800e+05, ...,
         1.04768000e+05,  4.98372300e+03, -1.00000000e+00],
       [ 1.73827092e+12,  1.04909800e+05,  1.05189300e+05, ...,
         1.04737000e+05,  4.83362100e+03, -1.00000000e+00],
       [ 1.73827926e+12,  1.05189200e+05,  1.04416800e+05, ...,
         1.04356100e+05,  4.86977900e+03, -1.00000000e+00]])

In [4]:
from custom_indicators.config import SIDE_ALL

side_features = df_features[SIDE_ALL]
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(44503, 2368)
(array([0, 1]), array([20772, 23731]))
(array([0, 1]), array([3872, 5029]))


In [5]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-18 23:13:17,121] A new study created in memory with name: no-name-a6f6d109-755b-4734-a090-042f17cc9485
[I 2025-03-18 23:13:50,703] Trial 0 finished with value: 0.8244636685735134 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 97, 'max_depth': 130, 'min_gain_to_split': 0.014752222678836285, 'min_data_in_leaf': 470, 'lambda_l1': 40.207358986972935, 'lambda_l2': 74.3810285439051, 'num_boost_round': 1188}. Best is trial 0 with value: 0.8244636685735134.


precision: 0.7971046282079404, recall: 0.7226088685623384, f1: 0.7580308719232374


[I 2025-03-18 23:14:07,183] Trial 1 finished with value: 0.8193383848883089 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 185, 'max_depth': 48, 'min_gain_to_split': 0.2246313230716316, 'min_data_in_leaf': 116, 'lambda_l1': 37.691387441560614, 'lambda_l2': 84.54294546444926, 'num_boost_round': 900}. Best is trial 0 with value: 0.8244636685735134.


precision: 0.7940133037694014, recall: 0.7120699940345994, f1: 0.7508124541356537


[I 2025-03-18 23:14:29,118] Trial 2 finished with value: 0.8293781398467401 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 217, 'max_depth': 799, 'min_gain_to_split': 0.5830611051043544, 'min_data_in_leaf': 81, 'lambda_l1': 91.64724581155298, 'lambda_l2': 69.64528645412508, 'num_boost_round': 1049}. Best is trial 2 with value: 0.8293781398467401.


...

precision: 0.7917301414581066, recall: 0.723404255319149, f1: 0.756026600166251


[I 2025-03-18 23:35:38,441] Trial 98 finished with value: 0.8361482738957023 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 118, 'max_depth': 184, 'min_gain_to_split': 0.5870519524126964, 'min_data_in_leaf': 445, 'lambda_l1': 3.884738174590418, 'lambda_l2': 84.88012849631176, 'num_boost_round': 208}. Best is trial 98 with value: 0.8361482738957023.


precision: 0.7993124194241513, recall: 0.7397096838337641, f1: 0.7683569141794898


[I 2025-03-18 23:35:46,004] Trial 99 finished with value: 0.8340288003135534 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 116, 'max_depth': 193, 'min_gain_to_split': 0.4619928895034619, 'min_data_in_leaf': 386, 'lambda_l1': 2.9074977174466454, 'lambda_l2': 75.93385067634634, 'num_boost_round': 195}. Best is trial 98 with value: 0.8361482738957023.


precision: 0.7983906046107003, recall: 0.7299661960628355, f1: 0.7626467227589072


In [7]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 118,
 'max_depth': 184,
 'min_gain_to_split': 0.5870519524126964,
 'min_data_in_leaf': 445,
 'lambda_l1': 3.884738174590418,
 'lambda_l2': 84.88012849631176,
 'num_boost_round': 208}

In [8]:
import lightgbm as lgb

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

In [9]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [10]:
import plotly.express as px
from sklearn.metrics import roc_auc_score

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8361482738957023


In [11]:
model.save_model("custom_indicators/models/model_side.txt")

# meta model

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_1h.parquet")
meta_label = np.load("data/label_meta_1h.npy")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features.isna().sum(axis=0).sort_values(ascending=False)

4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_comb_spectrum_pwr_14         0
1h_comb_spectrum_pwr_13         0
1h_comb_spectrum_pwr_12         0
1h_comb_spectrum_pwr_11         0
side_model_res                  0
Length: 11842, dtype: int64

In [2]:
df_features = df_features[364:]
meta_label = meta_label[364:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

(44503, 11842)
(44503, 7)


In [3]:
from custom_indicators.config import META_ALL

meta_features = df_features[META_ALL]
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(44503, 2368)
(array([0, 1]), array([ 6573, 29029]))
(array([0, 1]), array([2243, 6658]))


In [4]:
import lightgbm as lgb
import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1500),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.2, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return f1


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-19 00:44:23,633] A new study created in memory with name: no-name-ca49ebbb-316b-47a3-a6eb-9e9b165a5680
[I 2025-03-19 00:44:53,595] Trial 0 finished with value: 0.7764371894960965 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 134, 'max_depth': 613, 'min_gain_to_split': 0.7251385508347543, 'min_data_in_leaf': 38, 'lambda_l1': 67.05341593400176, 'lambda_l2': 68.96725597242481, 'num_boost_round': 377}. Best is trial 0 with value: 0.7764371894960965.


precision: 0.8173667607504566, recall: 0.7394112346049865, f1: 0.7764371894960965, auc: 0.6980845049522918, fbeta: 0.6906444399136196


[I 2025-03-19 00:45:13,308] Trial 1 finished with value: 0.7565031280869279 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 217, 'max_depth': 417, 'min_gain_to_split': 0.46400492765433193, 'min_data_in_leaf': 55, 'lambda_l1': 81.03298173472365, 'lambda_l2': 64.64485888894214, 'num_boost_round': 1323}. Best is trial 0 with value: 0.7764371894960965.


precision: 0.836976320582878, recall: 0.6901471913487534, f1: 0.7565031280869279, auc: 0.7111685672872728, fbeta: 0.6810327514416453


[I 2025-03-19 00:45:28,613] Trial 2 finished with value: 0.7558511948755852 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 386, 'max_depth': 538, 'min_gain_to_split': 0.9981203799665861, 'min_data_in_leaf': 127, 'lambda_l1': 51.741718975310725, 'lambda_l2': 78.46695668261113, 'num_boost_round': 869}. Best is trial 0 with value: 0.7764371894960965.


...

precision: 0.7742063492063492, recall: 0.8790928206668669, f1: 0.8233225488816992, auc: 0.6905823759027618, fbeta: 0.69507804301206


[I 2025-03-19 01:45:48,891] Trial 98 finished with value: 0.832074294822926 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 259, 'max_depth': 550, 'min_gain_to_split': 0.003290491674537901, 'min_data_in_leaf': 286, 'lambda_l1': 9.174939348312629, 'lambda_l2': 30.46562423684003, 'num_boost_round': 1287}. Best is trial 88 with value: 0.8348208155979678.


precision: 0.7724874533522069, recall: 0.9016221087413637, f1: 0.832074294822926, auc: 0.6865294477113604, fbeta: 0.6981637328977097


[I 2025-03-19 01:46:30,980] Trial 99 finished with value: 0.8229306566763271 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 375, 'max_depth': 522, 'min_gain_to_split': 0.12423035044771227, 'min_data_in_leaf': 287, 'lambda_l1': 12.376505501134988, 'lambda_l2': 49.69474903008256, 'num_boost_round': 1327}. Best is trial 88 with value: 0.8348208155979678.


precision: 0.7772733342235278, recall: 0.8742865725443076, f1: 0.8229306566763271, auc: 0.6861304894758192, fbeta: 0.6983111784239485


In [5]:
study.best_params

{'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 391,
 'max_depth': 390,
 'min_gain_to_split': 0.014946749575406013,
 'min_data_in_leaf': 200,
 'lambda_l1': 2.5130023850697687,
 'lambda_l2': 34.77548997954092,
 'num_boost_round': 1286}

In [6]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

In [7]:
import plotly.express as px
from sklearn.metrics import f1_score

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)
print(f1_score(meta_label_test, pred_label))

fig = px.histogram(res, nbins=100)
fig.show()

0.8348208155979678


In [8]:
model.save_model("custom_indicators/models/model_meta.txt")